In [10]:
import polars as pl

from src import constants

dataset = pl.scan_csv(constants.OUTPUT_DIR / "dataset.csv").collect()
dataset

session_id,yad_no,session_length,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,total_room_cnt_null_flg,wireless_lan_flg_null_flg,kd_stn_5min_null_flg,kd_bch_5min_null_flg,kd_slp_5min_null_flg,kd_conv_walk_5min_null_flg,mean_total_room_cnt_with_onsen_flg,std_total_room_cnt_with_onsen_flg,median_total_room_cnt_with_onsen_flg,mean_total_room_cnt_with_yad_type,std_total_room_cnt_with_yad_type,median_total_room_cnt_with_yad_type,mean_total_room_cnt_with_kd_bch_5min,std_total_room_cnt_with_kd_bch_5min,median_total_room_cnt_with_kd_bch_5min,mean_total_room_cnt_with_kd_stn_5min,std_total_room_cnt_with_kd_stn_5min,median_total_room_cnt_with_kd_stn_5min,mean_total_room_cnt_with_kd_bch_5min_right,std_total_room_cnt_with_kd_bch_5min_right,median_total_room_cnt_with_kd_bch_5min_right,mean_total_room_cnt_with_kd_conv_walk_5min,std_total_room_cnt_with_kd_conv_walk_5min,median_total_room_cnt_with_kd_conv_walk_5min,diff_total_room_cnt_with_onsen_flg
str,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""4a53607e6806d6…",719,1,0,600.0,1.0,0,1.0,1.0,1.0,1.0,0,0,0,1,1,0,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,492.294586
"""851a1663f06a51…",13292,1,0,525.0,1.0,0,1.0,1.0,1.0,1.0,0,0,0,1,1,0,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,417.294586
"""260bb3cc4b820b…",10095,1,0,2007.0,1.0,0,1.0,1.0,1.0,1.0,0,0,1,1,1,0,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,1899.294586
"""91badfb8a745f1…",755,2,0,912.0,1.0,0,1.0,1.0,1.0,1.0,0,0,0,1,1,0,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,804.294586
"""7205c78add17f5…",532,2,0,427.0,1.0,0,1.0,1.0,1.0,1.0,0,0,0,1,1,1,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,319.294586
"""0ac63c0fb237cf…",1091,2,0,289.0,1.0,1,1.0,1.0,1.0,1.0,0,0,1,1,1,1,52.679701,69.289168,29.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,236.320299
"""31c365dda81064…",1091,2,0,289.0,1.0,1,1.0,1.0,1.0,1.0,0,0,1,1,1,1,52.679701,69.289168,29.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,236.320299
"""479b5cf422c783…",719,4,0,600.0,1.0,0,1.0,1.0,1.0,1.0,0,0,0,1,1,0,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,492.294586
"""2423d790682d91…",12017,1,0,598.0,1.0,0,1.0,1.0,1.0,1.0,0,0,0,1,1,0,107.705414,119.321059,78.0,87.989802,107.357444,52.0,48.576613,81.279502,24.0,125.021743,126.811776,100.0,48.576613,81.279502,24.0,111.652157,119.851654,83.0,490.294586


In [11]:
preds = pl.scan_csv(constants.OUTPUT_DIR / "exp000" / "cv_fold0.csv").collect()
preds

session_id,yad_no,predict
str,i64,i64
"""3a355e5ea43f32…",4398,13549
"""3a355e5ea43f32…",4398,4913
"""3a355e5ea43f32…",4398,2118
"""3a355e5ea43f32…",4398,4601
"""3a355e5ea43f32…",4398,6334
"""3a355e5ea43f32…",4398,5624
"""3a355e5ea43f32…",4398,6987
"""3a355e5ea43f32…",4398,7036
"""3a355e5ea43f32…",4398,12954


In [12]:
preds["session_id"].unique().to_list()[:3]

['a6bf6cd42ede15cced9d63ab337b9e68',
 '97a508c3bfe2a8c07901226fcfaeeb53',
 '175ccdf0af4bd2f8ab2657e95d44ce7c']

In [17]:
train_label_df = pl.scan_csv(constants.INPUT_DIR / "train_label.csv").collect()
train_label_df

session_id,yad_no
str,i64
"""000007603d533d…",4101
"""0000ca043ed437…",8253
"""0000d4835cf113…",4863
"""0000fcda1ae1b2…",1652
"""000104bdffaaad…",96
"""00011afe25c343…",4823
"""000125c737df18…",10378
"""0001763050a10b…",10362
"""000178c4d4d567…",1227


In [21]:
see_targets = [
    "187cbc12d251ad3360f717c1cba7f1f2"
]
for target in see_targets:
    print(target)
    ds_this_target = dataset.filter(pl.col("session_id") == target)
    preds_this_target = preds.filter(pl.col("session_id") == target)
    print("DS: ", ds_this_target)
    print("PREDS: ", preds_this_target)
    yad_candidates = ds_this_target["yad_no"].to_list()
    pred = preds_this_target["predict"].to_list()
    label = train_label_df.filter(pl.col("session_id") == target)["yad_no"].to_list()[0]
    print("YAD CANDIDATES: ", yad_candidates)
    print("PREDS: ", pred)
    print("LABEL: ", label)
    print("Label in candidates: ", label in yad_candidates)
    print("")

187cbc12d251ad3360f717c1cba7f1f2
DS:  shape: (373, 36)
┌────────────┬────────┬────────────┬──────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ session_id ┆ yad_no ┆ session_le ┆ yad_type ┆ … ┆ mean_total ┆ std_total ┆ median_to ┆ diff_tota │
│ ---        ┆ ---    ┆ ngth       ┆ ---      ┆   ┆ _room_cnt_ ┆ _room_cnt ┆ tal_room_ ┆ l_room_cn │
│ str        ┆ i64    ┆ ---        ┆ i64      ┆   ┆ with_kd_co ┆ _with_kd_ ┆ cnt_with_ ┆ t_with_on │
│            ┆        ┆ i64        ┆          ┆   ┆ nv…        ┆ conv_…    ┆ kd_co…    ┆ sen_f…    │
│            ┆        ┆            ┆          ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│            ┆        ┆            ┆          ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════╪════════════╪══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 187cbc12d2 ┆ 11037  ┆ 1          ┆ 0        ┆ … ┆ 111.652157 ┆ 119.85165 ┆ 83.0      ┆ 495.29458 │
│ 51ad3360f7 ┆        ┆            ┆

In [22]:
label_in_candidates = 0
for target in dataset["session_id"].unique().to_list():
    ds_this_target = dataset.filter(pl.col("session_id") == target)
    preds_this_target = preds.filter(pl.col("session_id") == target)
    yad_candidates = ds_this_target["yad_no"].to_list()
    pred = preds_this_target["predict"].to_list()
    label = train_label_df.filter(pl.col("session_id") == target)["yad_no"].to_list()[0]
    if label in yad_candidates:
        label_in_candidates += 1

print(label_in_candidates / len(dataset["session_id"].unique().to_list()))